In [1]:
from __future__ import annotations
from langchain.globals import set_llm_cache
from langchain_community.cache import SQLiteCache
import os
import sys
import chromadb
# change the path to the backend directory
sys.path.append(os.path.join(os.path.dirname("."), '../../backend/'))

In [5]:
from modules.utils import get_all_metadata_from_openml, create_metadata_dataframe, load_config_and_device
from modules.llm import load_document_and_create_vector_store, setup_vector_db_and_qa

In [6]:
config = load_config_and_device("../../backend/config.json")
config["persist_dir"] = "../backend/data/chroma_db/"
config["data_dir"] = "../backend/data/"
config["type_of_data"] = "dataset"
config["training"] = True

# load the persistent database using ChromaDB
client = chromadb.PersistentClient(path=config["persist_dir"])
print(config)

[INFO] Finding device.
[INFO] Device found: cpu
{'rqa_prompt_template': 'This database is a list of metadata. Use the following pieces of context to find the relevant document. Answer only from the context given using the {question} given. If you do not know the answer, say you do not know. {context}', 'llm_prompt_template': 'The following is a set of documents {docs}. Based on these docs, please summarize the content concisely. Also give a list of main concepts found in the documents. Do not add any new information. Helpful Answer: ', 'num_return_documents': 50, 'embedding_model': 'BAAI/bge-large-en-v1.5', 'llm_model': 'qwen2:1.5b', 'num_documents_for_llm': 10, 'data_dir': '../backend/data/', 'persist_dir': '../backend/data/chroma_db/', 'testing_flag': False, 'ignore_downloading_data': False, 'test_subset_2000': False, 'data_download_n_jobs': 20, 'training': True, 'temperature': 0.95, 'top_p': 0.95, 'search_type': 'similarity', 'reranking': False, 'long_context_reorder': False, 'devic


## Manually

In [ ]:

# Download the data if it does not exist
openml_data_object, data_id, all_metadata = get_all_metadata_from_openml(
    config=config
)
# Create the combined metadata dataframe
metadata_df, all_metadata = create_metadata_dataframe(
    openml_data_object, data_id, all_metadata, config=config
)
# Create the vector store
vectordb = load_document_and_create_vector_store(
    metadata_df, config=config, chroma_client=client
)

## API

In [ ]:
qa = setup_vector_db_and_qa(
        config=config, data_type=config["type_of_data"], client=client
    )